In [1]:
from keras import backend as K
# Accuracy ne prenant pas en compte les charactères complétés

# Remove this when the cosineSimilarity will be added
def cosineSimilarity(h1, h2):
    return 2.3

def ignore_class_accuracy(to_ignore=2):
    def ignore_accuracy(y_true, y_pred):
        y_true_class = K.argmax(y_true, axis=-1)
        y_pred_class = K.argmax(y_pred, axis=-1)
 
        ignore_mask = K.cast(K.not_equal(y_pred_class, to_ignore), 'int32')
        matches = K.cast(K.equal(y_true_class, y_pred_class), 'int32') * ignore_mask
        accuracy = K.sum(matches) / K.maximum(K.sum(ignore_mask), 1)
        return accuracy
    return ignore_accuracy

# Fonction qui imitte le comportement du réseau de neurone
def get_hidden_state (word):
    prec = 0.005
    output = []
    
    for i, char in enumerate(word):
        if char == "a":
            prec = 0.02*(i+1) + prec
            output.append(prec)
        elif char == "b":
            prec = 0.03*(i+1) + prec
            output.append(prec)
        elif char == "e":
            prec = 0.005*(i+1) + prec
            output.append(prec)
        else:
            prec = 0.05*(i+1) + prec
            output.append(prec)
    return output



def get_data(filepath):

    inputs = []
    outputs = []
    with open(filepath, 'r') as f:
        lines = f.readlines()
    #print(lines[:3])


    max_length = 0

    for line in lines:
        res = ""
        isInput = True
        for symbol in line:
            if symbol in [',', '\n']:
                if isInput:
                    inputs.append(res)
                    max_length = len(res) if len(res) > max_length else max_length
                    res = ""
                    isInput = not isInput
                    continue
                else:
                    outputs.append(res)
            res += symbol
        #print(line)
    return inputs, outputs, max_length

def merging_checking(st1, st2, k):
    similarity = False
    consistency = False

    # for the similarity, we will merge state1 to state 2 if
    # If every input of state1 are in the input set of state2 and
    # If for each input state1 input set, we get the same output
    # from state1 and state2

    if set(st1._outTr.keys()) in set(st2._outTr.keys()):
        for char in list(st1._outTr.keys()):
            if set(st1._outTr[char].keys()) == set(st1._outTr[char].keys()):
                similarity = True

    # compute the cosine similarity of the two hidden state value
    # If it's greater than k, the consistency constraint in respected
    if cosineSimilarity(st1.hidden_state, st2.hidden_state) > k:
        consistency = True

    return similarity and consistency

def class_mapping(label, numb_class = 3):
    y_train = []
    for x in label:
        assert int(x) < numb_class
        y_train.append([int(i==int(x)) for i in range(numb_class)])
        
    return y_train

def tokenization(word, num_token = 4):
    x_train = []
    for x in word:
        if x == 'a':
            x_train.append(1)
        elif x == 'b':
            x_train.append(2)
        elif x == 'e':
            x_train.append(3)
        else:
            x_train.append(0)
    
    return x_train

def masking(word, pad_char = 'z'):
    return [x!=pad_char for x in word]

In [2]:
from model import Tagger 
import os
import numpy as np
from tqdm import trange
import tensorflow as tf
from tensorflow import keras

corpus, labels, max_length = get_data('dataset2.txt')
corpus_ = ["e"+x+"z"*(max_length-len(x)) for x in corpus]
labels_ = ["0"+x+"2"*(max_length - len(x)) for x in labels]
states = []

n_epochs = 10
batch_size = 10

x_train = np.array([tokenization(x) for x in corpus_])
y_train = np.array([class_mapping(x) for x in labels_])
mask = np.array([masking(x) for x in corpus_])

optimizer = keras.optimizers.Adam(learning_rate=0.01)

In [3]:
from sklearn.model_selection import train_test_split

# Division en jeu de test et jeu d'entrainement
X_train, X_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state = 42)

In [4]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import SimpleRNN
from keras.layers import Embedding
from keras.preprocessing import sequence
from sklearn.metrics import confusion_matrix

In [5]:
class obj:
    def __init__(self, x, y):
        self.x = x
        self.y = y

def change(obj):
    obj.x = 4

rer = obj(2,1)
print(rer.x)
change(rer)
print(rer.x)

2
4


In [10]:
n_epochs = 1
batch_size = 100
embedding_vector_length = 10

In [120]:
model = Sequential()
model.add(Embedding(4, embedding_vector_length))
model.add(SimpleRNN(10, return_sequences=True))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', ignore_class_accuracy(2)])
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, None, 10)          40        
                                                                 
 simple_rnn_8 (SimpleRNN)    (None, None, 10)          210       
                                                                 
 dense_4 (Dense)             (None, None, 3)           33        
                                                                 
Total params: 283
Trainable params: 283
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
history = model.fit(X_train, y_train, epochs=n_epochs, batch_size=batch_size, validation_split=0.2, verbose=1)

64/64 [==============================] - 38s 582ms/step - loss: 0.7074 - accuracy: 0.7652 - ignore_accuracy: 0.6348 - val_loss: 0.5537 - val_accuracy: 0.8284 - val_ignore_accuracy: 0.6811


In [115]:
list(history.history.keys())

['loss', 'accuracy', 'ignore_accuracy']

In [13]:
data = ['abbb', 'bbba', 'abab', 'aaaa', 'abba']
x_predict = np.array([tokenization(x) for x in data])
x_predict = tf.convert_to_tensor(x_predict)
model.predict(x_predict)

1/1 [==============================] - 1s 576ms/step


array([[[0.3826207 , 0.30852887, 0.30885047],
        [0.19417945, 0.5030848 , 0.30273584],
        [0.26252916, 0.3877717 , 0.34969908],
        [0.31407863, 0.3824414 , 0.30348   ]],

       [[0.21838854, 0.43559575, 0.3460157 ],
        [0.2548318 , 0.3848924 , 0.36027586],
        [0.259653  , 0.40548763, 0.33485934],
        [0.39601526, 0.30407542, 0.29990935]],

       [[0.3826207 , 0.30852887, 0.30885047],
        [0.19417945, 0.5030848 , 0.30273584],
        [0.43629703, 0.26407394, 0.29962897],
        [0.2424045 , 0.49866024, 0.25893533]],

       [[0.3826207 , 0.30852887, 0.30885047],
        [0.35004368, 0.37009576, 0.27986068],
        [0.3576279 , 0.37119833, 0.27117372],
        [0.42016873, 0.3390707 , 0.24076055]],

       [[0.3826207 , 0.30852887, 0.30885047],
        [0.19417945, 0.5030848 , 0.30273584],
        [0.26252916, 0.3877717 , 0.34969908],
        [0.49322522, 0.25169548, 0.25507924]]], dtype=float32)

In [15]:
def show_weights(model):
    print("Weights and biases of the layers after setting the new weights and biases: \n")
    for i, layer in enumerate(model.layers):
        if i == 0:
            continue
        print(layer.name)
        print("Weights")
        print("Shape: ",layer.get_weights()[0].shape,'\n',layer.get_weights()[0])
        print("Bias")
        print("Shape: ",layer.get_weights()[1].shape,'\n',layer.get_weights()[1],'\n')

In [16]:
model.layers[1].get_weights()

[array([[-0.42692873, -0.42289996,  0.4447741 , -0.4429699 , -0.17001548,
          0.11229646, -0.458028  , -0.01096553, -0.48083076,  0.5543523 ],
        [ 0.36340922, -0.18404362,  0.28009933,  0.20886461, -0.00743275,
          0.10764097,  0.15501681, -0.37760377,  0.29179093,  0.29020905],
        [-0.51575154,  0.40278122,  0.11884367,  0.42618102,  0.28357235,
         -0.51602376, -0.1604212 ,  0.09842136, -0.10567806,  0.02250358],
        [-0.17689845, -0.50512743, -0.5238936 ,  0.43742135,  0.05745095,
         -0.54296374, -0.56734043, -0.32235745,  0.1193771 , -0.08280785],
        [-0.03450042, -0.39813107, -0.23370494, -0.06218006,  0.27915868,
         -0.09502216, -0.39092863, -0.4534038 ,  0.4206866 , -0.36611205],
        [-0.23561552,  0.13641393,  0.5306019 , -0.1363112 , -0.6209317 ,
         -0.1086948 , -0.04420102,  0.3257237 , -0.5932869 ,  0.00980924],
        [-0.02885425,  0.3180489 , -0.11641473,  0.30534515, -0.26856256,
          0.30428317,  0.3672111

In [97]:
x = model.layers[0].get_weights()[0].tolist()
print(f'{type(model.layers[1].get_weights())}   {len(model.layers[1].get_weights())}')
print(f'{type(model.layers[1].get_weights()[2])}   {len(model.layers[1].get_weights()[2])}')
print(f'{type(model.layers[1].get_weights()[2][0])}   {model.layers[1].get_weights()[2][0]}')

<class 'list'>   3
<class 'numpy.ndarray'>   10
<class 'numpy.float32'>   0.004984821192920208


In [67]:
import json
def save_weights(model, filename):
    weights = []
    for layer in model.layers:
        w = []
        [w.append(x.tolist()) for x in layer.get_weights()]
        weights.append(w)
        
    with open(filename, 'w') as f:
        json.dump(weights, f)

In [125]:
model.get_weights()

[array([[-0.0276722 , -0.03214636,  0.04599688,  0.04878035, -0.03438361,
         -0.02427794,  0.02166354,  0.04073646, -0.02021405,  0.00705358],
        [-0.01297383, -0.03877324,  0.0131318 ,  0.01682233, -0.00295651,
         -0.04748244,  0.0160586 , -0.03909265, -0.03347943,  0.00540604],
        [-0.02497681,  0.01409762, -0.01605679,  0.00254904,  0.02868929,
         -0.04852531, -0.01363418, -0.04606549,  0.04233431, -0.0106622 ],
        [ 0.01290948,  0.04114536,  0.003541  , -0.00683626,  0.02310259,
         -0.00526153, -0.01384484, -0.0106406 , -0.03317811, -0.02855043]],
       dtype=float32),
 array([[-0.2904532 ,  0.33570272,  0.53752184, -0.50730807, -0.40321356,
         -0.36556786, -0.31149718,  0.18843633, -0.03316957, -0.3541723 ],
        [ 0.18768597, -0.4660169 ,  0.2519024 ,  0.13419265, -0.28947142,
          0.29118562,  0.16654259, -0.32439777, -0.52050495,  0.16858327],
        [-0.15505055, -0.34722754,  0.12369317, -0.08359307, -0.08467448,
        

In [124]:
import pickle
filename = 'weights.txt'
#save_weights(model, filename)
with open(filename, 'wb') as f:
    pickle.dump(model.layers, f)
#print(len(weights))

[<keras.layers.core.embedding.Embedding object at 0x000002870F948190>, <keras.layers.rnn.simple_rnn.SimpleRNN object at 0x000002871A8EEC10>, <keras.layers.core.dense.Dense object at 0x000002871B605210>]


In [123]:
with open(filename, 'rb') as f:
    rer = pickle.load(f)
rer

In [17]:
from copy import deepcopy

In [18]:
import numpy as np
rer = [np.random.randn(10,3), np.ones(3)]
len(rer[0])

10

In [19]:

x = np.array([[1]*len(model.layers[2].get_weights()[0][0])]*len(model.layers[2].get_weights()[0]))
print(np.asarray(x).shape)
y = np.array([0]*len(model.layers[2].get_weights()[1]))
print(y)
np.asarray(y).shape
model.layers[2].set_weights([x, y])

model.layers[2].get_weights()[1]

(10, 3)
[0 0 0]


array([0., 0., 0.], dtype=float32)

In [20]:
model.layers[1].get_weights()[2].shape

(10,)

In [111]:
print(len(model.layers[2].get_weights()))
print(f'{len(model.layers[2].get_weights()[0])}   {len(model.layers[2].get_weights()[0][0])}')
print(f'{len(model.layers[2].get_weights()[1])}   {model.layers[2].get_weights()[1][0]}')
model.layers[0].get_weights()

2
10   3
3   -0.004997094161808491


[array([[ 0.04087391,  0.01574843, -0.00389519, -0.04208458, -0.01874881,
          0.05217679, -0.03702322,  0.02232463,  0.01298681, -0.03275087],
        [ 0.02268491,  0.03186835, -0.02343684, -0.03106025,  0.03337424,
          0.00162261, -0.01158115,  0.01741067, -0.00934891,  0.01602577],
        [ 0.02666692,  0.04172854, -0.04967573,  0.01773047,  0.00267124,
          0.05301905,  0.01654921,  0.01729628,  0.02256451, -0.00616259],
        [-0.03813278,  0.00434956, -0.02140179,  0.02989013,  0.02575764,
          0.00664869,  0.01316514, -0.00521238, -0.00237513, -0.02024618]],
       dtype=float32)]

In [86]:
print(len(model.layers[1].get_weights()))
print(f'{len(model.layers[1].get_weights()[0])}   {len(model.layers[1].get_weights()[0][0])}')
print(f'{len(model.layers[1].get_weights()[1])}   {len(model.layers[1].get_weights()[1][0])}')
print(len(model.layers[1].get_weights()[2]))
#model.layers[1].get_weights()

3
10   10
10   10
10


In [119]:
model.pop()
print(len(model.layers))
model.summary()

2
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, None, 10)          40        
                                                                 
 simple_rnn_7 (SimpleRNN)    (None, None, 10)          210       
                                                                 
Total params: 250
Trainable params: 250
Non-trainable params: 0
_________________________________________________________________


In [90]:
x = np.array([[0]*10]*10)
x.shape

(10, 10)

In [109]:
weights = []
weights.append(np.array([[[0.0]*10]*4]))
weights.append([np.array([[0.0]*10]*10), np.array([[0.0]*10]*10), np.array([0.0]*10)])
weights.append([np.array([[0.0]*3]*10), np.array([0.0]*3)])
print(len(weights))
print(len(weights[0]))
print(len(weights[0][0]))
print(len(weights[0][0][0]))
print('\n')
print(len(weights[2]))
print(len(weights[2][0]))
print(len(weights[2][0][0]))
print(len(weights[2][1]))
np.asarray(weights).shape

3
1
4
10


2
10
3
3


C:\Users\Gael\AppData\Local\Temp\ipykernel_7376\641384744.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.asarray(weights).shape


(3,)

In [70]:
import json
import numpy as np

def load_weights(model, filename, return_states=True):
    with open(filename, 'r') as f:
        weights = json.load(f)
    
    model.layers[0].set_weights([np.array(weights[0][0])])
    model.layers[1].set_weights([np.array(weights[1][0]), np.array(weights[1][1]), np.array(weights[1][2])])

    if not return_states:
       model.layers[2].set_weights([np.array(weights[2][0]), np.array(weights[2][1])]) 

In [112]:
model1 = Sequential()
model1.add(Embedding(4, embedding_vector_length))
model1.add(SimpleRNN(10, return_sequences=True))
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', ignore_class_accuracy(2)])
#load_weights(model1, 'weights.txt')
model1.layers[0].set_weights(weights[0])
model1.layers[1].set_weights(weights[1])
print(model1.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, None, 10)          40        
                                                                 
 simple_rnn_6 (SimpleRNN)    (None, None, 10)          210       
                                                                 
Total params: 250
Trainable params: 250
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
data = ['abbb', 'bbbz', 'abab', 'aaaa', 'abba']
x_predict = np.array([tokenization(x) for x in data])
x_predict = tf.convert_to_tensor(x_predict)
ter = model1.predict(x_predict)
ter.shape

1/1 [==============================] - 0s 152ms/step


(5, 4, 10)

In [23]:
data = ['abbb'+'ab'*48, 'bbba'+'ab'*48, 'abab'+'ab'*48, 'aaaa'+'ab'*48]
x_predict = np.array([tokenization(x) for x in data])
states = model1.predict(x_predict)
states.shape

1/1 [==============================] - 0s 400ms/step


(4, 100, 10)

In [24]:
import json
def Tagger(n_tokens = 4, embedding_vector_length = 10, hidden_dim = 10, n_labels = 3, return_states = False):
    model = Sequential()
    model.add(Embedding(n_tokens, embedding_vector_length))
    model.add(SimpleRNN(hidden_dim, return_sequences=True))
    if not return_states:
        model.add(Dense(n_labels, activation='softmax'))
    return model

def load_weights(model, weights, return_states=False):
    if return_states:
        model.set_weights()

def save_weights(model, filepath):
    with open('data.json', 'w') as f:
        json.dump(model.layers, f)

def load_weights(model, filepath, only_state=False)):
    with open('data.json', 'w') as f:
        json.dump(model.layers, f)
    if not only_state:
        model.layers[0].set_weights([model.layers[0].get_weights()[0]])
        model.layers[1].set_weights([model.layers[1].get_weights()[0], model.layers[1].get_weights()[1], model.layers[1].get_weights()[2]])

model3 = Tagger()
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', ignore_class_accuracy(2)])
model3.summary()

SyntaxError: invalid syntax (2459124702.py, line 18)

In [126]:
corpus = ['ba', 'b', 'a', 'baa', 'a', 'baaa', 'aa', 'b', 'abaa', 'abb', 'bb']
labels = ['11', '1', '1', '110', '1', '1100', '10', '1', '1010', '101', '11']
corpus_ = [ "e"+x+"z"*(4-len(x)) for x in corpus]
labels_ = ["0"+x+"2"*(4-len(x)) for x in labels]
states = [get_hidden_state(x) for x in corpus_]
mask = [x!="z" for x in corpus_]


#print(corpus_)
#print(states)

x_train = tf.convert_to_tensor([tokenization(x) for x in corpus_])
train_sents = [tokenization(x) for x in corpus]
y_train = tf.convert_to_tensor([class_mapping(x) for x in labels_])
mask = tf.convert_to_tensor([masking(x) for x in corpus_])

In [127]:
x_train.numpy()

array([[2, 1, 0, 3, 3],
       [2, 1, 3, 3, 3],
       [2, 0, 3, 3, 3],
       [2, 1, 0, 0, 3],
       [2, 0, 3, 3, 3],
       [2, 1, 0, 0, 0],
       [2, 0, 0, 3, 3],
       [2, 1, 3, 3, 3],
       [2, 0, 1, 0, 0],
       [2, 0, 1, 1, 3],
       [2, 1, 1, 3, 3]])